In [1]:
# 掛載google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# 載入套件
# 只需要從資料庫抓取4個資訊：產品名稱、當日交易量、當日批發價格、模型預測的7日後價格
# 將抓取下來的表格，重新命名為以下操作的欄位名稱：price_today、amount_today、predict_price_7_d
from IPython.display import display
import pandas as pd
df_fruit = pd.read_csv('/content/drive/MyDrive/AI課程檔案/專題相關/line_chatbot/fake_data/fakedata_fruit.csv')
df_fruit = df_fruit.loc[:,['product', 'predict_price_today', 'predict_price_7_d', 'amount_1d_ago']]
df_fruit.columns = ['product','price_today','predict_price_7_d', 'amount_today']
df_fruit.head()

,product,price_today,predict_price_7_d,amount_today
0,香蕉,25.8,19.0,11197
1,鳳梨,39.5,38.9,64105
2,椪柑,15.4,10.4,36598
3,柳橙,14.5,21.9,41249
4,桶柑,11.4,18.8,59130


In [3]:
df_vegetable = pd.read_csv('/content/drive/MyDrive/AI課程檔案/專題相關/line_chatbot/fake_data/fakedata_vegetable.csv')
df_vegetable = df_vegetable.loc[:,['product', 'predict_price_today', 'predict_price_7_d', 'amount_1d_ago']]
df_vegetable.columns = ['product','price_today','predict_price_7_d', 'amount_today']
df_vegetable.head()


,product,price_today,predict_price_7_d,amount_today
0,蘿蔔,8.2,23.8,52990
1,胡蘿蔔,35.0,5.7,64737
2,其他根菜類,17.0,14.5,55100
3,薑,31.9,13.1,48413
4,芋,38.4,28.5,63785


以上是要等資料庫建置好後，到資料庫抓取需要的資料 / 
以下開始才是資料處理的code

* 目標一：蔬果價格排行表，篩選出10大熱門蔬果，抓取今日預測價格，並與1週後的價格做比較，預測漲跌
* 目標二：蔬果推薦：根據使用者的預算，推薦最划算的蔬果購買組合

In [4]:
# 讀取模型預測結果報表，抓取今日價格、以及1週後的價格比較
# 用7天後價格減去今天價格，正表示即將漲價、負表示即將跌價（尚待討論）
# 算出漲跌百分比：價差 / 今日預測價格
# 將今日價格換算成零售價
# 將結果新增到欄位內：價差、漲跌百分比、當日零售價
# 資料處理尚缺 批發價轉換零售價的部分，因此先以假設零售價為批發價的兩倍做為換算方式、等正式確認之後再補上

def price_count(df):
  df['price_difference'] = df['price_today']-df['predict_price_7_d']
  df['price_range'] = df['price_difference'] / df['price_today']
  icon_list = list()
  for i in df['price_difference']:
    if i > 0:
      icon_list.append('up')
    elif i < 0:
      icon_list.append('down')
    else:
      icon_list.append('flat')
  df['icon'] = icon_list
  df['retail_price_today'] = df['price_today'] * 2
  return df
df_fruit = price_count(df_fruit)
df_vegetable = price_count(df_vegetable)

In [5]:
# 將資料依照交易量、以及價差幅度進行排序、由大到小，前10筆作為10大交易量 的蔬菜 / 水果
def price_rank_sort(df):
  result_df = df.sort_values(by=['amount_today', 'price_range'], ascending=False, na_position='last', ignore_index=True)
  result_df = result_df.iloc[:10,:]
  return result_df

result_df_fruit = price_rank_sort(df_fruit)
result_df_vegetable = price_rank_sort(df_vegetable)

In [6]:
display(result_df_fruit)
display(result_df_vegetable)

,product,price_today,predict_price_7_d,amount_today,price_difference,price_range,icon,retail_price_today
0,楊桃,31.8,24.4,79556,7.4,0.232704,up,63.6
1,其他雜柑類,28.6,16.3,74925,12.3,0.430070,up,57.2
2,棗,32.4,32.7,73811,-0.3,-0.009259,down,64.8
3,番石榴,11.0,44.8,71908,-33.8,-3.072727,down,22.0
4,桃,49.4,34.0,69521,15.4,0.311741,up,98.8
5,蓮霧,30.2,42.8,69101,-12.6,-0.417219,down,60.4
6,梅,20.0,46.2,68028,-26.2,-1.310000,down,40.0
7,梨,11.7,34.3,67725,-22.6,-1.931624,down,23.4
8,荔枝,27.7,14.5,66694,13.2,0.476534,up,55.4
9,鳳梨,39.5,38.9,64105,0.6,0.015190,up,79.0


,product,price_today,predict_price_7_d,amount_today,price_difference,price_range,icon,retail_price_today
0,其他莖菜類,12.2,24.9,75311,-12.7,-1.040984,down,24.4
1,洋香瓜,33.5,38.1,73930,-4.6,-0.137313,down,67.0
2,其他葉菜類,21.3,11.3,70480,10.0,0.469484,up,42.6
3,花椰菜,15.9,8.5,69186,7.4,0.465409,up,31.8
4,洋蔥,14.0,17.3,69101,-3.3,-0.235714,down,28.0
5,西瓜,21.6,12.8,68143,8.8,0.407407,up,43.2
6,不結球白菜,7.2,34.6,67248,-27.4,-3.805556,down,14.4
7,胡蘿蔔,35.0,5.7,64737,29.3,0.837143,up,70.0
8,芋,38.4,28.5,63785,9.9,0.257812,up,76.8
9,蒜,6.9,38.7,61204,-31.8,-4.608696,down,13.8


1. 漲跌判斷是直接以價差決定，還是要有分區段（就像論文一樣價差超過一定程度才算漲or跌）
2. 零售價的轉換，會在哪一part處理，以及如何轉換？

In [7]:
import json
from pprint import pprint
# 計算完畢後，將結果放入flex_message版型的json檔，並儲存
# location到時候串接從資料庫抓取的時候，就需要設定是哪一區的location
location = '北區' 
def price_rank_json_template(df_fruit, df_veg, location):
  try:
    location = location
    fruit_name = list(df_fruit['product'])
    fruit_price = list(df_fruit['retail_price_today'])
    fruit_icon = list(df_fruit['icon'])
    vegetable_name = list(df_veg['product'])
    vegetable_price = list(df_veg['retail_price_today'])
    vegetable_icon = list(df_veg['icon'])

    icon_dict = {'up': 'https://i.imgur.com/9AgT25O.png',
                'down': 'https://i.imgur.com/Z02dd9W.png',
                'flat':'https://i.imgur.com/obmsbXZ.jpg'}
    # 存放 price_rank_json_template的位置
    json_model_file = '/content/drive/MyDrive/AI課程檔案/專題相關/line_chatbot/price_rank_json_template.txt'
    with open(json_model_file) as f:
      json_model = f.read()
    result = eval(json_model)
    pprint(result)
    json_file = json.dumps(result)
    # 將該json檔以當天日期檔名儲存
    # 儲存位置到時候看系統設定
    with open(f"/content/drive/MyDrive/AI課程檔案/專題相關/line_chatbot/price_rank_{location}.json",mode="w",encoding="utf-8") as f2:
      f2.write(json_file)
      return 'ok'
  except:
      return 'false'
print(price_rank_json_template(result_df_fruit, result_df_vegetable, location))

{'contents': [{'body': {'borderWidth': 'none',
                        'contents': [{'color': '#808A87',
                                      'margin': 'xs',
                                      'size': 'md',
                                      'text': '北區',
                                      'type': 'text'},
                                     {'margin': 'sm',
                                      'size': 'xl',
                                      'text': '熱門水果價格排行榜',
                                      'type': 'text',
                                      'weight': 'bold'},
                                     {'margin': 'xxl', 'type': 'separator'},
                                     {'contents': [{'contents': [{'decoration': 'underline',
                                                                  'flex': 0,
                                                                  'text': '今日價格',
                                                                  'type': 'te

以下為推薦菜單資料處理
暫定 5個區間
1. 200以內：買蔬菜3、買水果3、都買 2菜1果
2. 200-500：買蔬菜5、買水果5、都買 3菜2果
3. 500-1000：買蔬菜6、買水果6、都買 3菜3果
4. 1000以上：買蔬菜8、買水果8、都買 4菜4果
* 只推薦類型、不說明買的數量，目前邏輯處理不打算細算消耗預算的問題




In [8]:
# 以200元為例
# 將資料根據漲跌百分比做排序
# 依據指定的數字，篩選出產品

rule_dict = {0: { 'fruit': {'fruit': 3, 'veg': 0}, 'veg': {'fruit': 0, 'veg': 3}, 'veg+fruit': {'fruit': 1, 'veg': 2}},
 1: {'fruit': {'fruit': 5, 'veg': 0}, 'veg': {'fruit': 0, 'veg': 5}, 'veg+fruit': {'fruit': 2, 'veg': 3}},
 2: {'fruit': {'fruit': 6, 'veg': 0}, 'veg': {'fruit': 0, 'veg': 6}, 'veg+fruit': {'fruit': 3, 'veg': 3}},
 3: {'fruit': {'fruit': 8, 'veg': 0}, 'veg': {'fruit': 0, 'veg': 8}, 'veg+fruit': {'fruit': 4, 'veg': 4}}}

recommend_result_df = pd.DataFrame(columns=['budget', 'veg+fruit', 'fruit', 'veg'])
recommend_result_df['budget'] = ['200以內', '200-500', '500-1000', '1000以上']

df_fruit = df_fruit.sort_values(by=['price_range'], ascending=False, na_position='last')
df_vegetable = df_vegetable.sort_values(by=['price_range'], ascending=False, na_position='last')
fruit = list(df_fruit['product'])
veg = list(df_vegetable['product'])
recommend_fruit = df_fruit['product'].iloc[:3]
temp_dict = pd.DataFrame.from_dict(rule_dict)
temp_dict.T

for order, rule in rule_dict.items():
  for key, num_dict in rule.items():
    content = list()    
    f_num = num_dict['fruit']
    v_num = num_dict['veg'] 
    content.extend(fruit[:f_num])
    content.extend(veg[:v_num])
    recommend_result_df.loc[order, key] = content
recommend_result_df




,budget,veg+fruit,fruit,veg
0,200以內,"[晚崙西亞橙, 胡蘿蔔, 荸薺]","[晚崙西亞橙, 橄欖, 百香果]","[胡蘿蔔, 荸薺, 甘藍]"
1,200-500,"[晚崙西亞橙, 橄欖, 胡蘿蔔, 荸薺, 甘藍]","[晚崙西亞橙, 橄欖, 百香果, 荔枝, 其他雜柑類]","[胡蘿蔔, 荸薺, 甘藍, 其他果菜類, 薑]"
2,500-1000,"[晚崙西亞橙, 橄欖, 百香果, 胡蘿蔔, 荸薺, 甘藍]","[晚崙西亞橙, 橄欖, 百香果, 荔枝, 其他雜柑類, 木瓜]","[胡蘿蔔, 荸薺, 甘藍, 其他果菜類, 薑, 冬瓜]"
3,1000以上,"[晚崙西亞橙, 橄欖, 百香果, 荔枝, 胡蘿蔔, 荸薺, 甘藍, 其他果菜類]","[晚崙西亞橙, 橄欖, 百香果, 荔枝, 其他雜柑類, 木瓜, 可可椰子, 椪柑]","[胡蘿蔔, 荸薺, 甘藍, 其他果菜類, 薑, 冬瓜, 茭白筍, 毛豆]"


In [9]:
import json
from pprint import pprint

# 計算完畢後，將結果放入flex_message版型的json檔，並儲存
def recommend_list_template(df ,budget_num):
  try:
    budget_df = df.loc[budget_num]
    veg_fruit_content = list()
    fruit_content = list()
    veg_content = list()
    true = True 
    # json檔裡面有 true，在python裡面是True，但是用txt把檔案轉譯成python dict格式讀取出來
    # 如果沒有先指定 true = True，會把true當成一個變數
    for content, c in zip([veg_fruit_content, fruit_content, veg_content],['veg+fruit', 'fruit', 'veg']):
      for group in budget_df[c]:
        food = group
        text = {"type": "text",
                "text": food} # 文字檔格式
        content.append(text)

    # 存放 price_rank_json_template的位置
    template_file = '/content/drive/MyDrive/AI課程檔案/專題相關/line_chatbot/recommend_buy_template.txt'
    with open(template_file) as f:
      template_file = f.read()
    result = eval(template_file)
    pprint(result)
    json_file = json.dumps(result) # 轉換成json格式

    # 依照預算儲存json檔
    # 儲存位置到時候看系統設定
    with open(f"/content/drive/MyDrive/AI課程檔案/專題相關/line_chatbot/recommend_buy_{budget_num}.json",mode="w",encoding="utf-8") as f2:
      f2.write(json_file)
    return 'ok'

  except:
    return 'false'

# 將4種預算區間分別進行json格式的轉換
for num in range(4):
  recommend_list_template(recommend_result_df ,num)


{'contents': [{'body': {'contents': [{'margin': 'none',
                                      'size': 'xl',
                                      'text': '買蔬果',
                                      'type': 'text',
                                      'weight': 'bold',
                                      'wrap': True},
                                     {'contents': [{'text': '晚崙西亞橙',
                                                    'type': 'text'},
                                                   {'text': '胡蘿蔔',
                                                    'type': 'text'},
                                                   {'text': '荸薺',
                                                    'type': 'text'}],
                                      'layout': 'vertical',
                                      'margin': 'sm',
                                      'spacing': 'xs',
                                      'type': 'box'}],
                        'layout': 'vertica